In [188]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl


import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score,make_scorer
from sklearn.metrics import mean_absolute_error 



In [189]:
# Prepare the data
data = np.array([
  #     Temp    Cat  Time   Met-oil	  Yield
  #============================================
       [ 40. ,   3. , 105. ,  15. ,  59.8],
       [ 60. ,   3. , 105. ,  10.5,  92.6],
       [ 80. ,   1. , 105. ,  10.5,  88.2],
       [ 80. ,   5. , 105. ,  10.5,  60.6],
       [ 60. ,   5. , 150. ,  10.5,  85.2],
       [ 40. ,   5. , 105. ,  10.5,  59.4],
       [ 60. ,   3. , 105. ,  10.5,  92.6],
       [ 40. ,   3. , 150. ,  10.5,  58.8],
       [ 60. ,   3. , 150. ,  15. ,  75.6],
       [ 60. ,   1. , 105. ,   6. ,  69.6],
       [ 60. ,   5. , 105. ,  15. ,  76.6],
       [ 60. ,   3. , 150. ,   6. ,  78.2],
       [ 60. ,   3. ,  60. ,  15. ,  84. ],
       [ 60. ,   3. , 105. ,  10.5,  95. ],
       [ 60. ,   1. , 105. ,  15. ,  87. ],
       [ 60. ,   3. , 105. ,  10.5,  90.2],
       [ 80. ,   3. ,  60. ,  10.5,  81.8],
       [ 40. ,   3. ,  60. ,  10.5,  58.4],
       [ 60. ,   1. ,  60. ,  10.5,  89.6],
       [ 60. ,   5. , 105. ,   6. ,  65.6],
       [ 40. ,   3. , 105. ,   6. ,  59. ],
       [ 80. ,   3. , 105. ,   6. ,  86. ],
       [ 60. ,   3. ,  60. ,   6. ,  70. ],
       [ 60. ,   1. , 150. ,  10.5,  96.4],
       [ 80. ,   3. , 150. ,  10.5,  74.6],
       [ 80. ,   3. , 105. ,  15. ,  86.6],
       [ 60. ,   5. ,  60. ,  10.5,  79. ],
       [ 60. ,   3. , 105. ,  10.5,  92.6],
       [ 40. ,   1. , 105. ,  10.5,  79.8]
])

In [190]:
# Split the data into input features and target variable
input_data=data[:,:4]
target_data=data[:,4:5]

In [191]:
train_input, test_input, train_target, test_target= train_test_split(input_data, target_data, test_size=0.2, random_state=66)

print(f"Train Input: {train_input.shape[0]}")
print(f"Train Target: {train_target.shape[0]}")
print(f"Test Input: {test_input.shape[0]}")
print(f"Test Target: {test_target.shape[0]}")

Train Input: 23
Train Target: 23
Test Input: 6
Test Target: 6


In [192]:
sc_x=StandardScaler()
train_input=sc_x.fit_transform(train_input)
test_input=sc_x.transform(test_input)
input_data = sc_x.transform(input_data)

sc_y=StandardScaler()
train_target=sc_y.fit_transform(train_target)
test_target=sc_y.transform(test_target)
target_data=sc_y.transform(target_data)

In [193]:
def get_preds(test_target, test_target_predict):
    test_target=pd.DataFrame(test_target)
    test_target.rename(columns={0:'Actual'}, inplace=True)
    test_target_predict=pd.DataFrame(test_target_predict)
    test_target_predict.rename(columns={0:'Predicted'}, inplace=True)
    predictions=pd.concat([test_target, test_target_predict], axis=1)
    return predictions

In [194]:
# Prepare the data
# Define the function to create the ANN model
def create_model(units=100, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(units, input_shape=(4,), activation=activation))
    model.add(Dense(units, activation=activation))
    model.add(Dense(units, activation=activation))
    model.add(Dense(units, activation=activation))
    model.add(Dense(units, activation=activation))
    model.add(Dense(1))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model



# Create the KerasRegressor object
model = KerasRegressor(build_fn=create_model, verbose=0)



# Define the hyperparameters and their possible values
param_grid = {
    'units': [50, 70, 90, 110],
    'activation': ['relu', 'linear', "softmax"],
    'optimizer': ['adam', 'sgd', 'Adamax'],
}



# Create the GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit the model using grid search
grid_result = grid.fit(input_data, target_data)

# Print the best parameters and score
print("Best parameters: ", grid_result.best_params_)
print("Best score: ", grid_result.best_score_)


C:\Users\User\AppData\Local\Temp\ipykernel_2572\1107042200.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)


Best parameters:  {'activation': 'linear', 'optimizer': 'Adamax', 'units': 70}
Best score:  -0.7464790344238281


In [195]:
test_target_pred = grid.predict(test_input)

In [196]:
r2_score(test_target, test_target_pred)

0.1659231170940425

In [197]:
ann_params=grid.best_params_
ann_params

{'activation': 'linear', 'optimizer': 'Adamax', 'units': 70}